In [ ]:
pip install googletrans

In [ ]:
pip install tweet-preprocessor

In [ ]:
pip install tweepy

In [ ]:
from googletrans import Translator
import tweepy
import re
import os             
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
nltk.download('punkt')
import preprocessor as p
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer 




#input tweeter credentials#
CONSUMER_KEY = '3bAsLVoFl8HmlJBwJAB3hATYO'
CONSUMER_SECRET = 'mOyV7MHc22fhxGJdUrvVjqHY4cUcsApPBHu0tyiAzenWxCM1wB'
OAUTH_TOKEN = '1310696980859899909-1E9G87iXHIUOyPQcHJqxF4nsHptPac'
OAUTH_TOKEN_SECRET = 'YPXkDZz2K0dny0NfdS0clfJ4uCCKQVZn0hGaTIR9HV5tN'

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)


def removeNLTKStop(text):
    stop_words = set(stopwords.words('english')) 
    filtered_sentences = []
    w = []
    result_string=""
    
    text = re.sub(r'\.COM|[^a-zA-Z ]+|\s(?=&)|(?<!\w\w)\s+(?!\w\w)', '', text, 0, re.IGNORECASE)
    
    word_tokens = word_tokenize(text.lower()) 
    print(word_tokens)
    
    #Removed NLTK Stopwords 
    filtered_article = [w for w in word_tokens if not w in stop_words] 
    filtered_article = [] 
    
    ps = PorterStemmer() 
    stemmer = nltk.SnowballStemmer('english')
    lemmatizer = WordNetLemmatizer() 
    #for s in word_tokens:
     #   s = re.sub(r'\.COM|[^a-zA-Z ]+|\s(?=&)|(?<!\w\w)\s+(?!\w\w)', '', s, 0, re.IGNORECASE)
     #   result_string = result_string+s
        
    print(result_string)
    for w in word_tokens: 
        #w = ps.stem(w)
        #w = stemmer.stem(w)
        
     #   result_string = result_string+s
        w = lemmatizer.lemmatize(w,pos="v") 
        if w not in stop_words: 
            filtered_article.append(w)
    return " ".join(filtered_article)

def handle_emojis(tweet):
    # Smile -- :), : ), :-), (:, ( :, (-:, :')
    tweet = re.sub(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\))', ' EMO_POS ', tweet)
    # Laugh -- :D, : D, :-D, xD, x-D, XD, X-D
    tweet = re.sub(r'(:\s?D|:-D|x-?D|X-?D)', ' EMO_POS ', tweet)
    # Love -- <3, :*
    tweet = re.sub(r'(<3|:\*)', ' EMO_POS ', tweet)
    # Wink -- ;-), ;), ;-D, ;D, (;,  (-;
    tweet = re.sub(r'(;-?\)|;-?D|\(-?;)', ' EMO_POS ', tweet)
    # Sad -- :-(, : (, :(, ):, )-:
    tweet = re.sub(r'(:\s?\(|:-\(|\)\s?:|\)-:)', ' EMO_NEG ', tweet)
    # Cry -- :,(, :'(, :"(
    tweet = re.sub(r'(:,\(|:\'\(|:"\()', ' EMO_NEG ', tweet)
    return tweet

In [ ]:
def translateTweet(tweet):
    #translate
    translator = Translator()
    tweet = translator.translate(tweet) 
    return tweet

In [ ]:
def processTweet2(tweet):
    
  
    #Remove RT
    tweet = re.sub('RT','',tweet)
    #Convert to lower case
    tweet = tweet.lower()
    tweet = tweet.replace('-',' ', 1) 
    #tweet=re.sub(r'-(?:(?<!\b[0-9]{4}-)|(?![0-9]{2}(?:[0-9]{2})?\b))', ' ', tweet)

    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',tweet)
    #Convert @username to ''
    tweet = re.sub('@[^\s]+','',tweet)
    
    tweet = re.sub('u.s.','us',tweet)
    tweet = re.sub('united states','us',tweet)


    tweet = re.sub('u.k.','uk',tweet)
    tweet = re.sub('united kingdom','uk',tweet)


    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    print("here",tweet)
    #remove NLTK
    tweet = removeNLTKStop(tweet)
    
    
    #tweet = tweet.strip('\'"')
    tweet = re.sub('[^a-zA-Z0-9]', ' ', tweet)
    tweet.lstrip()


    #trim consecutive spaces
    tweet = re.sub(' +', ' ', tweet) 
    
    
    
    tweet = replaceTwoOrMore(tweet)

    
    # remove emoticons
    tweet = p.clean(tweet)
    return tweet    

In [ ]:
#strings =['W','A','R' N I N G']
#for s in strings:
#    s = re.sub(r'\.COM|[^a-zA-Z ]+|\s(?=&)|(?<!\w\w)\s+(?!\w\w)', '', s, 0, re.IGNORECASE)
#    print(s)

In [ ]:
def replaceTwoOrMore(s):
    #look for 2 or more repetitions of character and replace with the character itself
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    return pattern.sub(r"\1\1", s)
#end

In [ ]:
i=0
all_files = os.listdir("C:\\Users\\Trisha\\Downloads\\COVID-19-TweetIDs-master\\COVID-19-TweetIDs-master\\2020-01") 
for a in all_files: 
    print("Starting",a)
    f_name = 'C:\\Users\\Trisha\\Downloads\\COVID-19-TweetIDs-master\\COVID-19-TweetIDs-master\\2020-01\\'+a
    with open(f_name) as fp:
        line = fp.readline()
        while line:
            line = fp.readline()
            #print(line)
            try:
                api = tweepy.API(auth)
                tweet = api.get_status(line,tweet_mode="extended")
                if 'retweeted_status' in dir(tweet):
                    text=tweet.retweeted_status.full_text
                    #print("This is retweeted")
                else:
                    text=tweet.full_text
                    #print("This is original tweet")
                #print(tweet.user.location)
                #Change language
                if(tweet.lang!="en"):
                    text=translateTweet(text)
                #print(text)
                tweet=processTweet2(text)
                filename = 'C://Users//Trisha//Tweet_Data//'+str(i)+'.txt'
    
                f = open(filename, "w")

                f.write(tweet.lstrip())
                f.write("\n")
                i=i+1
                f.close()
                #print(tweet)
            except tweepy.TweepError:
                print("Cannot fetch tweets")
        
        print("Done")
